<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_spd_matrix
from numpy.linalg import inv, cond

In [2]:
df=pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Данные состроят из 5000 строк и 5 столбцов. 4 из них - признаки, а один - Страховые выплаты - целевой признак.  

In [5]:
# Количество пропусков
df.isnull().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [6]:
#дубликаты
df.duplicated().sum()

153

Пропущенных значений нет.153 дубликата.Вариативность данных низка по причине малого количества данных. Удалять дублиаты не будем. 

In [7]:
columns = ['Пол', 'Возраст',  'Зарплата',  'Члены семьи',  'Страховые выплаты']
print('Количество уникальных значений столбцов')
for i in columns:
     print( df[i].nunique())

Количество уникальных значений столбцов
2
46
524
7
6


Вывод:

5000 строк и 5 столбцов

Целевой признак: Страховые выплаты

Признаки: Пол, Возраст, Зарплата, Члены семьи

Данные пришли уже в обработанном состоянии.Предбработка не нужна.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится.

**Обоснование:** Докажем,что:
                             $$a = Xw = X1w = XPw1 = a1$$
   Для этого необходимо что бы :
                             $$w1 = P^-1w$$
   Представим новую матрицу признаков X1 как произведение старой X на матрицу P(обратимую):
                             $$X^i = Xw$$
   Подставим новое значение X1 в формулу w1:
                             $$w1=((XP)^{T}XP)^{-1}(XP)^{T}y$$
                             $$w1 = (X^{T}P^{T}XP)^{-1}$$
                             $$(P^{T}(X^{T}X)P)^{-1}P^{T}X^{T}y$$
                             $$w1 = P^{-1}(X^{T}X)^{-1}(P^{T})^{-1}P^{T}X^{T}y$$
   Так как  по условию P обратимая, то произведение P степени -1 и Р равно E (единичная матрица):                                                    $$w1+P^{-1}(X^{T}X)^{-1}EX^{T}y = P^{-1}({T}X)^{-1}X^{T}y$$
   Cправа получилась формула для w:
                             $$w1 = P^{-1}(X^{T}X)^{-1}X^{T}y = P^{-1}w$$
   
Подставим новое значение весов w1 в формулу для предсказаний линейной регрессии. Предсказания модели расчитываются по формуле: 
                            $$a = Xw$$
 Подставим в эту формулу значения X1 и w1 для того, чтобы на их основе вычислить предсказания модели a1:
                            $$a1 = X1w1 = XPP^{-1}w$$
 Так как P по условию обратимая, то произведение P и Pв -1 степени  снова равно E :
                            $$a1 = XPP^{-1}w = XEw = Xw = a$$
                            
 Предсказания а1 для матрицы признаков, умноженных на обратимую матрицу Р равны предсказаниям а.Параметры линейной регрессии в исходной задаче w и в преобразованной w1 связаны следующим образом:
                           $$w1 = P^{-1}w$$
  

## Алгоритм преобразования

**Алгоритм**

Для того чтобы закодировать данные пользователя, можно создать рандомно генирирующуюся матрицу которая имеет обратную себе матрицу. Таким образом домножая матрицу признаков на рандомную матрицу матрицу мы получим матрицу с закодированными значениями а для того чтобы получить обратно данные мы должны будем умножить закодированные данные на обратную рандомную матрицу

**Обоснование**



In [8]:
#Выделим из датасета набор признаков и целевой признак.
df_features = df.drop('Страховые выплаты', axis=1)
df_target = df['Страховые выплаты']

In [9]:
#Поделим признаки на test и train
features_train, features_test, target_train, target_test = train_test_split(df_features,\
                                                                            df_target,test_size=0.2,random_state=12345)

In [10]:
#Преобразуем выборки в матрицы

matrix_features_train = features_train.values
matrix_features_test = features_test.values

In [11]:
print(matrix_features_train.shape)
print(matrix_features_test.shape)

(4000, 4)
(1000, 4)


In [12]:
#Сделаем рандомную обратимую матрицу 4 на 4 по количеству признаков основной матрицы:

random_matrix = np.random.randn(4, 4)
random_matrix

array([[ 1.56367304,  0.73717011, -0.78955303, -0.58962849],
       [ 1.76252901,  0.39204528, -2.51081622, -1.06422601],
       [-1.25006842,  0.78552047,  0.33984895,  1.27187934],
       [ 1.11318838, -0.97933067, -1.16602743, -1.77833015]])

Полученная матрица является единичной, что значит случайная матрица обратима.

In [13]:
#Перемножим тестовую и тренировочную матрицы на получившуюся случайноую обратимую матрицу:

encoded_matrix_features_train = matrix_features_train.dot(random_matrix)
encoded_matrix_features_test = matrix_features_test.dot(random_matrix)

In [14]:
#Для проверки попробуем перемножить признаки train на обратную случайной матрицу:

decoded_matrix_features_train = encoded_matrix_features_train.dot(np.linalg.inv(random_matrix))

## Проверка алгоритма

Создадим модель для матриц.

In [15]:
model = LinearRegression()#Линейная регрессия

model.fit(features_train, target_train)
predistions = model.predict(features_test)
print('R2_score на обычных данных:', r2_score(target_test, predistions))

R2_score на обычных данных: 0.4117683956770434


In [16]:
model.fit(encoded_matrix_features_train, target_train)
predicted_encoded = model.predict(encoded_matrix_features_test)
#encoded_r2 = model.score(encoded_matrix_features_test, target_test)
print('R2_score на кодированных данных:', r2_score(target_test, predicted_encoded))

R2_score на кодированных данных: 0.4117683956769197


In [17]:
rezume=[ r2_score(target_test, predistions),r2_score(target_test, predicted_encoded)]
df_rezume=pd.DataFrame(rezume,index=['R2_score на обычных данных','R2_score на кодированных данных'])
df_rezume.set_axis(['r2_score'],axis='columns',inplace=True)
df_rezume


,r2_score
R2_score на обычных данных,0.411768
R2_score на кодированных данных,0.411768


 Вывод: Мы проверили качества моделей Линейной регрессии с помощью метрики R2 на данных - сначала на исходных, затем умноженных на обратимую матрицу,т,е, кодированных, размер которой равен числу признаков. Показатели метрики совпали. Можно сделать вывод, что алгоритм работает.